<p><markdown translate="no" mathjax="" class="ng-tns-c364-40"><h1 id="processamento-de-linguagem-natural-pln"><strong>Processamento de Linguagem Natural (PLN)</strong></h1>
<h2 id="modelo-de-classificação"><strong>Modelo de Classificação</strong></h2>
<h3 id="índice">Índice</h3>
<ul>
<li>1) Objetivos</li>
<li>2) Metodologia</li>
<li>3) Introdução</li>
<li>4) Count vectorization</li>
<li>5) TF-IDF</li>
<li>6) Classificadores</li>
</ul>
<h3 id="1-objetivos"><strong>1) Objetivos</strong></h3>
<p>Este material tem como objetivo apresentar conceitos acerca de:</p>
<ul>
<li>Extração de features de um texto;</li>
<li>Construção de um modelo simples para classificação;</li>
<li>Treino e teste do modelo.</li>
</ul>
<h3 id="2-metodologia"><strong>2) Metodologia</strong></h3>
<p>Com os conceitos abordados nos últimos dois materiais, vamos construir agora um modelo capaz de classificar se uma mensagem (ou e-mail) é spam ou não-spam (ham).</p>
<p>Modelos deste tipo utilizam amplamente as técnicas de PLN que discutimos anteriormebte, e eles têm enorme importância no mundo real, sendo usados por todos os provedores de e-mail e também em problemas de prevenção a fraudes!</p>
<p>A base de dados que utilizaremos está hospedada no seguinte link, que contém maiores detalhes sobre ela:</p>
<ul>
<li>Classificação de Spam: <a href="http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection">http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection</a>;</li>
</ul>
<h3 id="3-introdução"><strong>3) Introdução</strong></h3>
<p>Os algoritmos de Machine Learning não aceitam textos não estruturados. Para processar os textos, é preciso fazer a extração de features, para transformá-los em variáveis numéricas.</p>
<p>Para isso, utilizaremos o modelo de <strong>Bag of Words</strong>: converte o texto em uma matriz com a quatidade de vezes que cada palavra aparece no documento.</p>
<ul>
<li>Além disso, vamos usar discutir o uso das classes do sklearn como: <code>Counter Vectorization</code>, <code>Term-Frequency</code> e <code>Inverse Document Frequency</code></li>
</ul>
<h3 id="4-count-vectorization"><strong>4) Count vectorization</strong></h3>
<p>Para podermos usar textos não estruturados em algoritmos de Machine Learning, fazemos a chamada "vetorização" dos textos.</p>
<p>Um tipo de vetorização é a "count vectorization", que consiste em contar a quantidade de vezes que cada palavra aparece no texto.</p>
<p>Vejamos exemplos práticos em Python.</p>
<p>Primeiramente, vamos ler os dados que utilizaremos, analisá-los brevemente, e fazer o train-test split.</p>
<pre class=" language-python"><code class=" language-python"><span class="token comment"># baixa os dados</span>
<span class="token comment"># esta é a base de dados de mensagens spam ou não spam (ham)</span>
<span class="token comment"># !wget http://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip</span>
<span class="token comment"># !unzip smsspamcollection.zip</span>

<span class="token comment"># importa o pandas para manipulação de dados</span>
<span class="token keyword">import</span> pandas <span class="token keyword">as</span> pd
<span class="token comment"># lê os dados e os armazena no dataframe "df"</span>
df <span class="token operator">=</span> pd<span class="token punctuation">.</span>read_csv<span class="token punctuation">(</span><span class="token string">'./SMSSpamCollection'</span><span class="token punctuation">,</span> sep<span class="token operator">=</span><span class="token string">'\t'</span><span class="token punctuation">,</span> names<span class="token operator">=</span><span class="token punctuation">[</span><span class="token string">'label'</span><span class="token punctuation">,</span> <span class="token string">'message'</span><span class="token punctuation">]</span><span class="token punctuation">)</span>

<span class="token comment"># examina as 5 primeiras linhas do dataframe</span>
df<span class="token punctuation">.</span>head<span class="token punctuation">(</span><span class="token punctuation">)</span>
<span class="token comment"># []label    message</span>
<span class="token comment">#0    ham    Go until jurong point, crazy.. Available only ...</span>
<span class="token comment">#1    ham    Ok lar... Joking wif u oni...</span>
<span class="token comment">#2    spam    Free entry in 2 a wkly comp to win FA Cup fina...</span>
<span class="token comment">#3    ham    U dun say so early hor... U c already then say...</span>
<span class="token comment">#4    ham    Nah I don't think he goes to usf, he lives aro...</span>

<span class="token comment"># checando se existe valores vazios na base de dados</span>
df<span class="token punctuation">.</span>isnull<span class="token punctuation">(</span><span class="token punctuation">)</span><span class="token punctuation">.</span><span class="token builtin">sum</span><span class="token punctuation">(</span><span class="token punctuation">)</span>
<span class="token comment"># [] label      0</span>
<span class="token comment"># message    0</span>
<span class="token comment"># dtype: int64</span>
<span class="token comment"># não há falta de dados!</span>

<span class="token comment"># verificar o balanceamento do dataset</span>
df<span class="token punctuation">[</span><span class="token string">'label'</span><span class="token punctuation">]</span><span class="token punctuation">.</span>value_counts<span class="token punctuation">(</span><span class="token punctuation">)</span>
<span class="token comment"># [] ham     4825</span>
<span class="token comment"># spam     747</span>
<span class="token comment"># Name: label, dtype: int64</span>
<span class="token comment"># temos muito mais mensagens classificadas como "ham" do que "spam"!</span>
<span class="token comment"># o desbalanceamento de classes deve ser tratado para não influenciar a performance do algoritmo</span>

<span class="token comment"># importa a função de train-test split</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>model_selection <span class="token keyword">import</span> train_test_split

<span class="token comment"># armazenando as features (no caso, as mensagens)</span>
X <span class="token operator">=</span> df<span class="token punctuation">[</span><span class="token string">'message'</span><span class="token punctuation">]</span>
<span class="token comment"># armazenando o target (nesse caso, a label que classifica a mensagem em spam ou ham)</span>
y <span class="token operator">=</span> df<span class="token punctuation">[</span><span class="token string">'label'</span><span class="token punctuation">]</span>

<span class="token comment"># faz o train-test split com 20% na base de teste</span>
X_train<span class="token punctuation">,</span> X_test<span class="token punctuation">,</span> y_train<span class="token punctuation">,</span> y_test <span class="token operator">=</span> train_test_split<span class="token punctuation">(</span>X<span class="token punctuation">,</span> y<span class="token punctuation">,</span> test_size<span class="token operator">=</span><span class="token number">.20</span><span class="token punctuation">,</span> random_state<span class="token operator">=</span><span class="token number">52</span><span class="token punctuation">)</span></code></pre>
<h3 id="countvectorizer"><strong>CountVectorizer</strong></h3>
<p>Todas as etapas de pré-processamento que envolvem a vetorização do texto (como tokenização, filtro de stop words, etc.) podem ser feitas através da classe "CountVectorizer" do sklearn:</p>
<pre class=" language-python"><code class=" language-python"><span class="token comment"># iporta a classe "CountVectorizer" do sklearn</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>feature_extraction<span class="token punctuation">.</span>text <span class="token keyword">import</span> CountVectorizer
<span class="token comment"># instancia a classe</span>
count_vect <span class="token operator">=</span> CountVectorizer<span class="token punctuation">(</span><span class="token punctuation">)</span>

<span class="token comment"># constrói a matriz de vocabulário e conta o número de palavras</span>
<span class="token comment"># essa será a matriz de features, já vetorizadas!</span>
X_train_counts <span class="token operator">=</span> count_vect<span class="token punctuation">.</span>fit_transform<span class="token punctuation">(</span>X_train<span class="token punctuation">)</span>

X_train_counts<span class="token punctuation">.</span>shape
<span class="token comment"># [] (4457, 7763)</span>
<span class="token comment"># temos uma matriz com 4457 linhas e 7763 colunas</span></code></pre>
<h3 id="5-tf-idf"><strong>5) TF-IDF</strong></h3>
<p>No contexto de PLN, há dois termos muito importantes:</p>
<ul>
<li><strong>Term frequency (TF)</strong>: é a frequência com a qual determinado termo aparece no texto;</li>
<li><strong>Inverse document frequency (IDF)</strong>: é o inverso da frequência com a qual determinado termo aparece entre diferentes documentos de um corpus (conjunto de documentos);</li>
</ul>
<p>Multiplicando estes dois fatores, temos a quantidade <strong>TF-IDF</strong>, que é uma estatística que quantidica a importância de determinada palavra em um documento, quando se considera diferentes documentos.</p>
<ul>
<li>TF-IDF = term frequency * inverse document frequency</li>
</ul>
<p>Ou seja, se uma palavra é extremamente comum em um documento, mas não for tão comum em outros documentos similares, isto pode significar que há algo de particular e pouco genérico neste documento em questão.</p>
<p>Mas, se a palavra for comum em todos os documentos considerados, este pode ser um indicativo de que a palavra de fato é importante em todo o contexto considerado!</p>
<p>Uma vetorização que leva em consideração o TF-IDF, é, portanto, mais sensível à importância relativa das palavras no texto!</p>
<p>Vamos incorporar esta estatística à nossa base:</p>
<pre class=" language-python"><code class=" language-python"><span class="token comment"># importa a classe "TfidfTransformer" do sklearn</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>feature_extraction<span class="token punctuation">.</span>text <span class="token keyword">import</span> TfidfTransformer
<span class="token comment"># instancia a classe</span>
tfidf_transformer <span class="token operator">=</span> TfidfTransformer<span class="token punctuation">(</span><span class="token punctuation">)</span>

<span class="token comment"># criamos a versão tf-idf das features de treino vetorizadas</span>
x_train_tfidf <span class="token operator">=</span> tfidf_transformer<span class="token punctuation">.</span>fit_transform<span class="token punctuation">(</span>X_train_counts<span class="token punctuation">)</span>

<span class="token comment"># no sklearn existe ainda a classe "TfidfVectorizer" que faz o CountVectorizer + TfidfTransformer</span>
<span class="token comment"># importando a classe "TfidfVectorizer"</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>feature_extraction<span class="token punctuation">.</span>text <span class="token keyword">import</span> TfidfVectorizer
<span class="token comment"># instanciando a classe</span>
vectorizer <span class="token operator">=</span> TfidfVectorizer<span class="token punctuation">(</span><span class="token punctuation">)</span>

<span class="token comment"># aplicando a vetorização + tf_idf diretamente às features de treino originais</span>
x_train_tfidf <span class="token operator">=</span> vectorizer<span class="token punctuation">.</span>fit_transform<span class="token punctuation">(</span>X_train<span class="token punctuation">)</span>
</code></pre>
<h3 id="6-classificadores"><strong>6) Classificadores</strong></h3>
<p>Uma vez que pré-processamos os dados não-estruturados, transformando-os em vetores numéricos, podemos criar o modelo de Machine Learning para classificar os e-mails em spam ou ham!</p>
<p>O modelo que construiremos será um classificador de Support Vector Machines, com a classe "LinearSVC" do sklearn</p>
<pre class=" language-python"><code class=" language-python"><span class="token comment"># importa a classe do modelo classificador de SVM</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>svm <span class="token keyword">import</span> LinearSVC
<span class="token comment"># instancia a classe</span>
clf <span class="token operator">=</span> LinearSVC<span class="token punctuation">(</span><span class="token punctuation">)</span>

<span class="token comment"># fita o modelo com os dados de teste vetorizados</span>
clf<span class="token punctuation">.</span>fit<span class="token punctuation">(</span>x_train_tfidf<span class="token punctuation">,</span> y_train<span class="token punctuation">)</span>

<span class="token comment"># Alternativamente, podemos criar um pipeline para fazer</span>
<span class="token comment"># o pré-processamento bem como a criação do modelo </span>
<span class="token comment"># diretamente nos dados originais</span>

<span class="token comment"># importa a classe Pipeline</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>pipeline <span class="token keyword">import</span> Pipeline
<span class="token comment"># cria o pipeline com o vetorizador tfidf e com o modelo svm classificador</span>
text_clf <span class="token operator">=</span> Pipeline<span class="token punctuation">(</span><span class="token punctuation">[</span>
    <span class="token punctuation">(</span><span class="token string">'tfidf'</span><span class="token punctuation">,</span>TfidfVectorizer<span class="token punctuation">(</span><span class="token punctuation">)</span><span class="token punctuation">)</span><span class="token punctuation">,</span>
    <span class="token punctuation">(</span><span class="token string">'clf'</span><span class="token punctuation">,</span> LinearSVC<span class="token punctuation">(</span><span class="token punctuation">)</span><span class="token punctuation">)</span>
<span class="token punctuation">]</span><span class="token punctuation">)</span>

<span class="token comment"># fita o pipeline aos dados originais</span>
text_clf<span class="token punctuation">.</span>fit<span class="token punctuation">(</span>X_train<span class="token punctuation">,</span>y_train<span class="token punctuation">)</span>
<span class="token comment"># faz predições na base de teste</span>
<span class="token comment"># note que o pipeline garante que a vetorização das features</span>
<span class="token comment"># da base de teste também será feita!</span>
predictions <span class="token operator">=</span> text_clf<span class="token punctuation">.</span>predict<span class="token punctuation">(</span>X_test<span class="token punctuation">)</span>

<span class="token comment"># visualiza as predições</span>
predictions
<span class="token comment"># [] array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype=object)</span></code></pre>
<p>Uma vez feitas as predições, avaliamos a performance de nosso modelo:</p>
<pre class=" language-python"><code class=" language-python"><span class="token comment"># importa as funções de avaliação de performance</span>
<span class="token keyword">from</span> sklearn<span class="token punctuation">.</span>metrics <span class="token keyword">import</span> confusion_matrix<span class="token punctuation">,</span> classification_report<span class="token punctuation">,</span> accuracy_score

<span class="token comment"># exibe a matriz de confuçao</span>
confusion_matrix<span class="token punctuation">(</span>y_test<span class="token punctuation">,</span> predictions<span class="token punctuation">)</span>
<span class="token comment"># [] array([[964,   1],</span>
<span class="token comment">#       [ 16, 134]])</span>

<span class="token comment"># exibe o classification report</span>
<span class="token keyword">print</span><span class="token punctuation">(</span>classification_report<span class="token punctuation">(</span>y_test<span class="token punctuation">,</span> predictions<span class="token punctuation">)</span><span class="token punctuation">)</span>
<span class="token comment">#              precision    recall  f1-score   support</span>
<span class="token comment">#</span>
<span class="token comment">#         ham       0.98      1.00      0.99       965</span>
<span class="token comment">#        spam       0.99      0.89      0.94       150</span>
<span class="token comment">#</span>
<span class="token comment">#    accuracy                           0.98      1115</span>
<span class="token comment">#   macro avg       0.99      0.95      0.97      1115</span>
<span class="token comment">#weighted avg       0.98      0.98      0.98      1115</span>

<span class="token comment"># exibe a acurácia</span>
<span class="token keyword">print</span><span class="token punctuation">(</span>accuracy_score<span class="token punctuation">(</span>y_test<span class="token punctuation">,</span> predictions<span class="token punctuation">)</span><span class="token punctuation">)</span>
<span class="token comment"># [] 0.9847533632286996</span></code></pre>
<p>Nosso modelo está muito bom, com um f1-score ponderado de 98%! Realmente, uma performance admirável para um modelo tão simples, o que pode ser devido ao correto pré-processamento dos dados!</p>
<p>Por fim, podemos fazer alguns testes explícitos de nosso modelo em algumas mensagens de teste:</p>
<pre class=" language-python"><code class=" language-python">text_clf<span class="token punctuation">.</span>predict<span class="token punctuation">(</span><span class="token punctuation">[</span><span class="token string">'Congratulations! You are a new winner!'</span><span class="token punctuation">]</span><span class="token punctuation">)</span>
<span class="token comment"># [] array(['spam'], dtype=object)</span>

text_clf<span class="token punctuation">.</span>predict<span class="token punctuation">(</span><span class="token punctuation">[</span><span class="token string">'Hello, how are you?'</span><span class="token punctuation">]</span><span class="token punctuation">)</span>
<span class="token comment"># [] array(['ham'], dtype=object)</span></code></pre>
<p>Muito bem, o modelo acertou!</p>
</markdown></p>